# 1. Ознакомиться с описанием датасета, определить целевую переменную (target) и основную задачу прогнозной модели.

Датасет в себе содержит общие данные о клиентах (возраст, семейное положение, образование и тд), данные связанные с последним  контактом с клиентом (способ связи, длительность и тд), эконономический контекст (значение ставки Euribor 3M, индекс потребительских цен и тд) и другие. 

Целевой переменной является ‘y’, принимающий значения ‘yes’ или ‘no’. Название переменной не является информативным, однако из описания датасета мы узнаем, что она означает положит клиент срочный депозит или нет.

Основная задача прогнозной модели по имеющимся данным предсказать положит ли клиент срочный депозит.

# 2. Выполнить этапы разведочного анализа, указанные в блокноте.

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv("../input/bank-additional-full.csv", sep=';')
df.head()

# Каков средний возраст холостых/незамужних клиентов?

Холостых:

In [ ]:
df.groupby('marital')['age'].mean()['single']

Незамужних:

In [ ]:
df[(df['marital'] == 'single') | (df['marital'] == 'divorced')]['age'].mean()

Незамужним считаем любого клиента, семейное положение которого определено и не 'married'.

# В какой день недели (признак **day_of_week**) чаще всего звонили клиентам, отказавшимся от депозита?

In [ ]:
df[df['y'] == 'no']['day_of_week'].value_counts().keys()[0]

Метод Series.value_counts() среди своих параметров, заданных по умолчанию, имеет такие: sort=True, ascending=False.
Это означает, что возвращаемый методом Series будет отсортирован по насчитаному value в порядке убывания.
Таким образом, в элементе с индексом 0 гарантированно лежит максимум.

# Постройте инфографику по признакам **marital** и **y**. Какие выводы можно сделать?

In [ ]:
sns.heatmap(pd.crosstab(df['marital'], df['y']), cmap="BuPu", annot=True, cbar=True);

Из инфографики видно, что целевая переменная 'y' в целом чаще принимает значения 'no'.

Также можем видеть что срочные депозиты чаще кладут разведенные клиенты,чем женатые или холостые. О клиентах с неопределнным семейным положением данных слишком мало, чтоб что-либо утрверждать. 

In [ ]:
chi2_contingency(pd.crosstab(df['marital'], df['y']))

Вычислив коэффициент $\chi^2$ видим, что p-value < порогового значения (0.05), а значит признаки marital и y взаимосвязаны.

# Имеется ли связь между наличием кредита (**default**) у клиента и результатом? Постройте инфографику.

Так как оба этих признака категориальные, вычислим коэффициент $\chi^2$.

In [ ]:
default_y_crosstab = pd.crosstab(df['default'], df['y'])

chi2_contingency(default_y_crosstab)

Значения p-value в данном случае очень маленькое (< 0.05), что свидетельствует о наличии статистической связи между наличием кредита у клиента и результатом. О характере и силе связи коэффициент $\chi^2$ информации не дает.

In [ ]:
sns.heatmap(default_y_crosstab, cmap="BuPu", annot=True, cbar=True);

# Визуализируйте информацию об уровне образования в зависимости от среднего возраста. Есть ли статистически значимая взаимосвязь между ними?

In [ ]:
plt.figure(figsize=(10, 10))
df.groupby('education')['age'].mean().plot(kind='bar') 
plt.ylabel('Age')
plt.show();

Так как один из признаков количественный, а другой ранговый, можно вычислить коэффициент корреляции Спирмена.

In [ ]:
stats.spearmanr(df.groupby('education')['age'].mean()[df['education']], df['education'])

Так как p-value меньше порогового значения, статистическая взаимосвязь есть. Эта взаимосвязь обратная и достаточно слабая.

# Что можно сказать о связи между длительностью контакта (признак **duration**) и возрастом клиента? Найдите коэффициент корреляции.

Так как оба признака количественные можно использовать коэффициент корелляции Пирсона.

In [ ]:
stats.pearsonr(df['duration'], df['age'])

В данном случае p-value больше порогового значения, а значит принимаем гипотезу о том, что связь между признаками статистически не является значимой.

# Как связаны признаки **education** и **housing**?

In [ ]:
df['housing_bin']=(df['housing']=='yes').astype(int)
df.head()

In [ ]:
stats.spearmanr(df['housing_bin'], df['education'])

P-value достаточно мало, а значит статистическая связь есть. Коэффициент коррелляции положительный, что свидетельствует о прямой взяимосвязи, однако очень слабой.

# Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

1. Если что-то кажется ерундой, читай документацию
2. Искать корреляцию в категориальных данных печально
3. Большинство клиентов предпочитают не класть срочный депозит
4. В датасете нет ни одного признака, который бы сильно или умеренно влиял на целевую переменную
5. Некоторые признаки имеют очень неинформативные названия, что очень усложняет понимание происходящего


# 3. Выполнить творческое задание: попытаться найти признаки (“фичи”), которые будут наиболее полезными для предсказания target-переменной. Использовать любые доступные инструменты разведочного анализа (вычисления,визуализация, корреляционный анализ).

In [ ]:
df.head()

Для того, чтоб было удобно рассмотреть все зависимости построим матрицу корелляций. Однако, целевая переменная y категориального типа, а значит в эту матрицу не войдет. Добавим бинарный признак y_bin, где 1 соответствует значению 'yes', а 0 - 'no'.

In [ ]:
df['y_bin']=(df['y']=='yes').astype(int)

Рассмотрим визуализацию матрицы корреляций Спирмена.

In [ ]:
df.corr(method='pearson')

In [ ]:
sns.heatmap(df.corr(method='pearson'));

* Прямое направление

    Видим, что длительность телефонного звонка с клиентом имеет самую сильную взаимосвязь с тем, положит ли этот клиент депозит в банк. Однако эта взаимосвязь все равно считается слабой.

    Связь чуть слабее с количеством контактов до этого клиента (previous).
    
    Очень слабая взаимосвязь между целевым признаком и такими признаками как consumer confidence index, наличие заема на жилье(housing) и возрастом.
    
* Обратная взаимосвязь

    Наиболее сильная с признаком pdays и количеством сотрудников (тоже считается слабая связь), чуть слабее на целевую переменную влияют признаки employment variation rate, euribor 3 month rate и еще слабее consumer price index.
    
    Почти незначительная взаимосвязь с признаками campaign.





Осталось исследовать признаки: job, marital, education, default, loan, contact, month, day_of_week и poutcome.

In [ ]:
print("Job:", stats.spearmanr(df['y_bin'], df['job']))
print("Marital:", stats.spearmanr(df['y_bin'], df['marital']))
print("Education:", stats.spearmanr(df['y_bin'], df['education']))
print("Loan:", stats.spearmanr(df['y_bin'], df['loan']))
print("Contact:", stats.spearmanr(df['y_bin'], df['contact']))
print("Month:", stats.spearmanr(df['y_bin'], df['month']))
print("Day_of_week:", stats.spearmanr(df['y_bin'], df['day_of_week']))
print("Poutcome:", stats.spearmanr(df['y_bin'], df['poutcome']))

Среди всех полученых значений можно заметить, что p-value для loan превышанет пороговое значение, а значит loan не имеет статистической связи с y.

Job, marital, education, day_of_week имеет несущественную прямую взаимосвязь, а poutcome более сильную прямую связь.

Month имеет несущественную обратную взаимосвязь, а фича contact более сильную.

# Вывод

Наиболее полезными для предсказания target-переменной будут признаки:
* duration
* previous
* pdays
* employment variation rate
* euribor 3 month rate